In [1]:
import transformers
import torch, math, einops
from src import Tokenizer, linear_unidirectional_graph_maker

In [2]:
tokenizer = Tokenizer('gpt2')
pretraied = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [3]:
from src.GPT2 import GPT2, GPT2_Encoder, GPT2_LM_Head

encoder=GPT2_Encoder()
decoder=GPT2_LM_Head()

model=GPT2(encoder, decoder, tokenizer)

model.load_from_original(pretraied)

In [4]:
def split_heads(tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

In [29]:
x=torch.randint(0, 50257, (1, 10))
x=torch.randn([1,13,768])

sequence_lenght=17
batch_size=1
n_heads=13
d_Embedding=64*n_heads
Q=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])
K=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])
V=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])


out, att = pretraied.transformer.h[0].attn._attn(Q,K,V,None,None)
#out=out.permute(1, 0, 2).contiguous().view(batch_size, sequence_lenght, d_Embedding)
out.shape,att.shape
out=out.permute(0,2,1,3).view(sequence_lenght,n_heads,d_Embedding//n_heads)



In [30]:
att_pretrained = einops.einsum(Q,K,'... s e, ... t e -> ... s t')/math.sqrt(K.size(-1))
alignments=att_pretrained
query_length, key_length = Q.size(-2), K.size(-2)

causal_mask = pretraied.transformer.h[0].attn.bias[:, :, key_length - query_length : key_length, :key_length].to(torch.bool)
mask_value = -torch.tensor(float('inf'))
att_pretrained = torch.where(causal_mask, att_pretrained, mask_value)
alignments=att_pretrained
att_pretrained = torch.nn.functional.softmax(att_pretrained, dim=-1)
att_pretrained = att_pretrained.view(n_heads,sequence_lenght,sequence_lenght)

(att_pretrained==att).all()



tensor(True)

In [31]:
from src.transformerMP import attention_message

graph_maker=linear_unidirectional_graph_maker(40)

edge_index=graph_maker(sequence_lenght)
senders, receivers = edge_index

Q1=Q.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)
K1=K.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)
V1=V.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)

#o,a=attention_message(Q,K,V,edge_index,0)
#o.view(sequence_lenght,d_Embedding)

N=20

i,j=receivers[N],senders[N]
print((Q1[receivers]*K1[senders]).sum(dim=-1)[N]/math.sqrt(K.size(-1)))

print(alignments.view(n_heads,sequence_lenght,sequence_lenght)[:,i,j])

tensor([-0.7951,  0.0413, -0.2327, -0.5042, -0.1293, -0.5635,  0.1652, -0.6069,
        -0.9542, -1.1916, -1.6909,  1.3947,  0.1134])
tensor([-0.7951,  0.0413, -0.2327, -0.5042, -0.1293, -0.5635,  0.1652, -0.6069,
        -0.9542, -1.1916, -1.6909,  1.3947,  0.1134])


In [128]:
from src.transformerMP import normalize_strength
from math import sqrt
def attention_message(Q:torch.Tensor,
                      K:torch.Tensor,
                      V:torch.Tensor,
                      edge_index:torch.Tensor,
                      att_dropout=0.1
                      ):

    senders, receivers = edge_index
    N,h,d=K.shape   

    #Q.K^T 
    att=(Q[receivers]*K[senders]).sum(dim=-1)/sqrt(d)

    #softmax    
    att = torch.exp(att+3-att.max()) #could be done in-plase using the function att.exp_() if memory is a bootleneck
    attention = normalize_strength(att, receivers, N, h)

    #Dropout
    #att=attention_dropout(attention, att_dropout)

    #softmax*V
    att = einops.einsum(attention,V[senders],' ... , ... c -> ... c')
    out = torch.zeros_like(V,device=V.device)

    return out.index_add(0,receivers,att), attention #could be done in-place using the function out.index_add_()

In [129]:
out_function,a=attention_message(Q1,K1,V1,edge_index,0)

i=3
print(edge_index[:,receivers==j])
print(att_pretrained[i,j:,j])
print(a[senders==j,i])
asd=True
for j in range(att_pretrained.shape[-1]):
    asd= asd and torch.allclose(att_pretrained[:,j:,j],a.permute(1,0)[:,senders==j],1e-4,1e-4)
asd
    

tensor([[0, 1, 2, 3],
        [3, 3, 3, 3]])
tensor([0.2588, 0.2823, 0.1009, 0.0054, 0.3157, 0.0963, 0.0137, 0.0299, 0.1241,
        0.1702, 0.0130, 0.1355, 0.1829, 0.0291])
tensor([0.2588, 0.2823, 0.1009, 0.0054, 0.3157, 0.0963, 0.0137, 0.0299, 0.1241,
        0.1702, 0.0130, 0.1355, 0.1829, 0.0291])


True

In [130]:
V=V.view(n_heads,sequence_lenght,d_Embedding//n_heads)
att_pretrained.shape,V.shape
print(att_pretrained.shape,V.shape)
AV_pretrained=einops.einsum(att_pretrained,V,'h i j, h j e -> h i j e')
print(AV_pretrained.shape)

if not (AV_pretrained.sum(-2)==torch.matmul(att_pretrained,V)).all(): #true
    print (False)

print(a.shape,V1[senders].shape)
dsa=einops.einsum(a,V1[senders],'..., ... e -> ... e')
dsa.shape==V1[senders].shape==(152, n_heads, d_Embedding//n_heads) #True

N=9
i,j=senders[N],receivers[N]
print(torch.allclose(dsa[N],AV_pretrained[:,j,i],1e-3,1e-3))

torch.Size([13, 17, 17]) torch.Size([13, 17, 64])
torch.Size([13, 17, 17, 64])
torch.Size([153, 13]) torch.Size([153, 13, 64])
True


In [134]:
out_pretrained=AV_pretrained.sum(-2).permute(1,0,2)
dsum=torch.zeros_like(V1).index_add(0,receivers,dsa)

print(out_pretrained.shape,dsum.shape)
print(torch.allclose(out_pretrained,dsum,1e-3,1e-3))
out,_=
torch.allclose(out,out_function,1e-3,1e-3)
#print((dsa[N]==asd[:,j,i]))

torch.Size([17, 13, 64]) torch.Size([17, 13, 64])
True
torch.Size([17, 13, 64])


True

In [36]:

print(a.shape,att.shape)
N=1

i,j=senders[N],receivers[N]

print(i,j)

a[N],att_pretrained[:,j,i]

torch.Size([153, 13]) torch.Size([1, 13, 17, 17])
tensor(0) tensor(1)


(tensor([0.5362, 0.6045, 0.3577, 0.1286, 0.2299, 0.4903, 0.2952, 0.7785, 0.0389,
         0.3664, 0.3424, 0.8425, 0.5351]),
 tensor([0.5362, 0.6045, 0.3577, 0.1286, 0.2299, 0.4903, 0.2952, 0.7785, 0.0389,
         0.3664, 0.3424, 0.8425, 0.5351]))

In [113]:
i=5

print(edge_index[:,i])
attn_weights=att_pretrained.view(n_heads,sequence_lenght,sequence_lenght)
print(a[i])
attn_weights[:,receivers[i],senders[i]]

tensor([2, 2])
tensor([0.3432, 0.5988, 0.2493, 0.1185, 0.2133, 0.1271, 0.5732, 0.2741, 0.4261,
        0.5167, 0.0223, 0.0487])


tensor([0.4503, 0.0660, 0.3172, 0.8236, 0.3810, 0.2807, 0.1164, 0.0755, 0.0994,
        0.2715, 0.6308, 0.7693, 0.5859])

In [11]:
torch.isclose(o,out,1e-3,1e-3)

RuntimeError: The size of tensor a (64) must match the size of tensor b (768) at non-singleton dimension 2

In [15]:
x=x.view(x.shape[1:])
graph_maker=linear_unidirectional_graph_maker(40)
edge_index=graph_maker(x.shape[0])

model.transformer_blocks[0].attention_block(x,edge_index)

tensor([[ 2.0409,  1.3881,  4.1935,  ..., -0.0925,  0.1177,  0.0000],
        [ 1.0567, -1.3801,  1.3112,  ..., -0.0530, -0.0237,  0.1608],
        [-4.0899,  1.3798,  1.4433,  ...,  0.0746,  0.0000,  0.3461],
        ...,
        [ 0.1670, -0.1714, -0.1629,  ...,  0.1357, -0.0147,  0.0000],
        [ 0.1670, -0.1714, -0.1629,  ...,  0.1357, -0.0147,  0.0315],
        [ 2.9507,  3.0202,  0.3895,  ...,  0.1534,  0.1628,  0.0112]],
       grad_fn=<MulBackward0>)

In [25]:
from src import linear_unidirectional_graph_maker

sample_text = "Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry"
x=tokenizer(sample_text)

graph_maker=linear_unidirectional_graph_maker(20)
edge_index=graph_maker(len(x))
#x,edge_index
tokenizer.decode(x)

'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry'

In [26]:
out=model(x,edge_index)
out.max(dim=-1)

torch.return_types.max(
values=tensor([0.5262, 0.9352, 0.6346, 0.9176, 0.9545, 0.4873, 0.6251, 0.3462, 0.5225,
        0.6778, 0.4525, 0.8533, 0.7261, 0.6538, 0.7236, 0.5359, 0.4798, 0.5364],
       grad_fn=<MaxBackward0>),
indices=tensor([  262,   262, 47490,   262,   262, 47490,   262,   262,   262,   262,
        47490,   262,   262,   262, 47490,   262, 47490,   262]))

In [20]:
tokenizer.decode(out.argmax(dim=-1))

'��� the the��� the the indo the the the the confir the the the the, the'

In [3]:
import torch
def count_parameters(model):
    """Count the number of parameters in a model."""
    return sum([p.numel() for p in model.parameters()])

conv = torch.nn.Conv1d(8,32,1)
print(count_parameters(conv))
# 288

linear = torch.nn.Linear(8,32)
print(count_parameters(linear))
# 288

print(conv.weight.shape)
print(conv.bias.shape)
# torch.Size([32, 8, 1])
print(linear.weight.shape)
print(linear.bias.shape)
# torch.Size([32, 8])

# use same initialization
linear.weight = torch.nn.Parameter(conv.weight.squeeze(2))
linear.bias = torch.nn.Parameter(conv.bias)

288
288
torch.Size([32, 8, 1])
torch.Size([32])
torch.Size([32, 8])
torch.Size([32])
